In [32]:
# packages
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('max_colwidth', None)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from imblearn.under_sampling import RandomUnderSampler
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt

# Data Preprocessing

In [56]:
#preparing dataframe
#Save the dataframes in the same directory as the notebook for the below reads to work. 
#df2018 = pd.read_parquet('Combined_Flights_2018.parquet', engine='pyarrow')
#df2019 = pd.read_parquet('Combined_Flights_2019.parquet', engine='pyarrow')
#df2020 = pd.read_parquet('Combined_Flights_2020.parquet', engine='pyarrow')
df2021 = pd.read_parquet('Combined_Flights_2021.parquet', engine='pyarrow')
#df2022 = pd.read_parquet('Combined_Flights_2022.parquet', engine='pyarrow')

#frames = [df2018, df2019,df2020,df2021,df2022]
#df=pd.concat(frames)

# I was struggling to get code to run with all data, dropped down to a single year here. BH
df = df2021

In [57]:
#cleaning dataset
df.dropna(inplace=True)

df = df.select_dtypes(include=['number'])
# bad features below that are technically numbers, but useless including groups, IDs
df = df.drop('Year', axis=1) #we may want to bring this one back in if analyzing multiple years
df = df.drop('DivAirportLandings', axis=1)
df = df.drop('OriginAirportID', axis=1)
df = df.drop('OriginAirportSeqID', axis=1)
df = df.drop('OriginCityMarketID', axis=1)
df = df.drop('DestAirportID', axis=1)
df = df.drop('DestAirportSeqID', axis=1)
df = df.drop('DestCityMarketID', axis=1)
df = df.drop('DOT_ID_Operating_Airline', axis=1)
df = df.drop('DOT_ID_Marketing_Airline', axis=1)
df = df.drop('OriginStateFips', axis=1)
df = df.drop('Flight_Number_Marketing_Airline', axis=1)
df = df.drop('Flight_Number_Operating_Airline', axis=1)
df = df.drop('OriginWac', axis=1)
df = df.drop('DestWac', axis=1)
df = df.drop('DestStateFips', axis=1)
df = df.drop('DepartureDelayGroups', axis=1)
df = df.drop('ArrDelay', axis=1) #removing this since having a delay over 15 will automatically trigger the classification
df = df.drop('ArrivalDelayGroups', axis=1)
df = df.drop('DistanceGroup', axis=1)
df = df.drop('DepDelay', axis=1)#removing since having a departure deplay people can assume arrival will also be delayed
df = df.drop('DepDelayMinutes', axis=1)#removing since having a departure deplay people can assume arrival will also be delayed
df = df.drop('DepDel15', axis=1) #removing since having a departure deplay people can assume arrival will also be delayed
df = df.drop('Quarter', axis=1) #removing this due to high correlation between quarter and month
df = df.drop('ActualElapsedTime', axis=1) #removing due to high correlation with other variables
df = df.drop('CRSElapsedTime', axis=1) #removing due to high correlation with other variables
df = df.drop('WheelsOff', axis=1) #removing due to high correlation with other variables
df = df.drop('WheelsOn', axis=1) #removing due to high correlation with other variables
df = df.drop('Distance', axis=1) #removing due to high correlation with other variables
df = df.drop('CRSArrTime', axis=1) #removing due to high correlation with other variables
df = df.drop('CRSDepTime', axis=1) #removing due to high correlation with other variables

In [35]:
#df.info()

In [36]:
#completing correlation matrix to determine highly correlated variables that we can remove CS
#correlation_matrix = df.corr()
#threshold = 0.8  # You can adjust this threshold as needed
#highly_correlated_pairs = []
#for i in range(len(correlation_matrix.columns)):
    #for j in range(i + 1, len(correlation_matrix.columns)):
        #if abs(correlation_matrix.iloc[i, j]) > threshold:
           # feature1 = correlation_matrix.columns[i]
            #feature2 = correlation_matrix.columns[j]
            #highly_correlated_pairs.append((feature1, feature2, correlation_matrix.iloc[i, j]))
#for feature1, feature2, correlation in highly_correlated_pairs:
    #print(f"Highly correlated pair: {feature1} and {feature2} (Correlation: {correlation:.2f})")

In [37]:
#splitting data for test/training
X = df.drop('ArrDel15', axis=1)
y = df['ArrDel15']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)   #creating training data set, 80% of the data


In [38]:
#undersampling not delayed class. I am undersampling before normalizing since the class imbalance is severe.
undersampler = RandomUnderSampler(sampling_strategy=0.5, random_state=42)
X_train_undersampled, y_train_undersampled = undersampler.fit_resample(X_train, y_train)


In [39]:
#X_train_undersampled.info()

In [40]:
#normalizing the data
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train_undersampled)
X_test_normalized = scaler.transform(X_test)

# Random Forest Classifier

In [41]:

# create random forest classifier
rfc = RandomForestClassifier(random_state=1)

# grid search paramters
param_grid = {'n_estimators': range(1, 200, 75),
              'max_depth': range(5, 16, 5),
              'min_samples_split': range(10, 41, 15),
              'min_samples_leaf': range(20, 50, 15)}
              

# create grid search with cross validation of 5 folds
grid_search = GridSearchCV(rfc, param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)

# perform grid search on training data
grid_search.fit(X_train_normalized, y_train_undersampled)

# model with best parameters-P0[]
best_rf_classifier = grid_search.best_estimator_

Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=10, n_estimators=1; total time=   2.0s
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=10, n_estimators=1; total time=   2.0s
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=10, n_estimators=1; total time=   2.1s
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=10, n_estimators=1; total time=   2.1s
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=10, n_estimators=1; total time=   2.2s
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=25, n_estimators=1; total time=   2.1s
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=25, n_estimators=1; total time=   2.1s
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=25, n_estimators=1; total time=   1.9s
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=25, n_estimators=1; total time=   1.9s
[CV] END max_depth=5, min_samples_leaf

# Random Forest Classifier with ADABoost

In [42]:
ada_rf_classifier = AdaBoostClassifier(base_estimator=rfc, random_state=1)

param_grid = {'n_estimators': range(1, 200, 75),
              'max_depth': range(5, 16, 5),
              'min_samples_split': range(10, 41, 15),
              'min_samples_leaf': range(20, 50, 15)}

grid_search = GridSearchCV(rfc, param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=10, n_estimators=1; total time=   5.8s
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=10, n_estimators=1; total time=   6.3s
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=10, n_estimators=1; total time=   6.2s


/Users/geordy/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=10, n_estimators=1; total time=   6.5s
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=10, n_estimators=1; total time=   6.4s
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=25, n_estimators=1; total time=   4.8s
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=25, n_estimators=1; total time=   5.1s
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=25, n_estimators=1; total time=   4.4s
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=25, n_estimators=1; total time=   4.4s
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=25, n_estimators=1; total time=   4.2s
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=10, n_estimators=76; total time= 2.5min
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=10, n_estimators=76; total time= 2.5min
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=10, n_estimators=76; total time= 2.5m

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=1), n_jobs=-1,
             param_grid={'max_depth': range(5, 16, 5),
                         'min_samples_leaf': range(20, 50, 15),
                         'min_samples_split': range(10, 41, 15),
                         'n_estimators': range(1, 200, 75)},
             scoring='accuracy', verbose=2)

In [44]:
# Calculate accuracy, precision, recall, and F1 score 
best_rfada_classifier = grid_search.best_estimator_
y_test_pred_rcfada = best_rfada_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_test_pred_rcfada)
precision = precision_score(y_test, y_test_pred_rcfada)
recall = recall_score(y_test, y_test_pred_rcfada)
f1 = f1_score(y_test, y_test_pred_rcfada)

print("RFCAda Accuracy: ", accuracy)
print("RFCAda Precision: ", precision)
print("RFCAda Recall: ", recall)
print("RFCAda F1: ", f1)

RFCAda Accuracy:  1.0
RFCAda Precision:  1.0
RFCAda Recall:  1.0
RFCAda F1:  1.0


In [65]:
#feature importance
importances = best_rfada_classifier.feature_importances_
indices = np.argsort(importances)[::-1]
print("Feature ranking:")
feature_names = df.columns.tolist()
for f in range(df.shape[1]):
    print(f"{f+1}. Feature {feature_names[indices[f]]} - {importances[indices[f]]}")

Feature ranking:
1. Feature ArrDelayMinutes - 0.9359579666749331


IndexError: list index out of range

In [ ]:
#removing unimportant features
df = df.drop('DayofMonth', axis=1)
df = df.drop('DayOfWeek', axis=1)
df = df.drop('AirTime', axis=1)
df = df.drop('Month', axis=1)
X = df.drop('ArrDel15', axis=1)
y = df['ArrDel15']

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)   #creating training data set, 80% of the data
undersampler = RandomUnderSampler(sampling_strategy=0.5, random_state=42)
X_train_undersampled, y_train_undersampled = undersampler.fit_resample(X_train, y_train)
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train_undersampled)
X_test_normalized = scaler.transform(X_test)

rfcADA2 = AdaBoostClassifier(base_estimator=rfc, random_state=1)

# grid search paramters
param_grid = {'n_estimators': range(1, 200, 75),
              'max_depth': range(5, 16, 5),
              'min_samples_split': range(10, 41, 15),
              'min_samples_leaf': range(20, 50, 15)}
              

# create grid search with cross validation of 5 folds
grid_search1 = GridSearchCV(rfc, param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)

# perform grid search on training data
grid_search1.fit(X_train_normalized, y_train_undersampled)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=10, n_estimators=1; total time=   1.5s
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=10, n_estimators=1; total time=   1.6s
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=10, n_estimators=1; total time=   1.6s
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=10, n_estimators=1; total time=   1.7s
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=10, n_estimators=1; total time=   1.7s
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=25, n_estimators=1; total time=   1.5s
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=25, n_estimators=1; total time=   1.6s
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=25, n_estimators=1; total time=   1.5s
[CV] END max_depth=5, min_samples_leaf=20, min_samples_split=25, n_estimators=1; total time=   1.5s
[CV] END max_depth=5, min_samples_leaf

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=1), n_jobs=-1,
             param_grid={'max_depth': range(5, 16, 5),
                         'min_samples_leaf': range(20, 50, 15),
                         'min_samples_split': range(10, 41, 15),
                         'n_estimators': range(1, 200, 75)},
             scoring='accuracy', verbose=2)

In [62]:
# Calculate accuracy, precision, recall, and F1 score 
best_rfada2_classifier = grid_search1.best_estimator_
y_test_pred_rcfada = best_rfada2_classifier.predict(X_test_normalized)
accuracy = accuracy_score(y_test, y_test_pred_rcfada)
precision = precision_score(y_test, y_test_pred_rcfada)
recall = recall_score(y_test, y_test_pred_rcfada)
f1 = f1_score(y_test, y_test_pred_rcfada)

print("RFCAda2 Accuracy: ", accuracy)
print("RFCAda2 Precision: ", precision)
print("RFCAda2 Recall: ", recall)
print("RFCAda2 F1: ", f1)

RFCAda2 Accuracy:  1.0
RFCAda2 Precision:  1.0
RFCAda2 Recall:  1.0
RFCAda2 F1:  1.0


In [66]:
#feature importance
importances = best_rfada2_classifier.feature_importances_
indices = np.argsort(importances)[::-1]
print("Feature ranking:")
feature_names = df.columns.tolist()
for f in range(df.shape[1]):
    print(f"{f}. Feature {feature_names[indices[f]]} - {importances[indices[f]]}")

Feature ranking:
0. Feature ArrDelayMinutes - 0.9258126207229774
1. Feature TaxiOut - 0.031111538460810863
2. Feature ArrTime - 0.02013351409422484
3. Feature DepTime - 0.01691891128269718
4. Feature TaxiIn - 0.006023415439289815


IndexError: index 5 is out of bounds for axis 0 with size 5

In [ ]:
#both random forest's models are over fitting.